# Imports

In [2]:
!pip install pyspark

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 
import os
import pyspark

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import to_date, date_format

from sklearn.model_selection import train_test_split
from category_encoders.one_hot import OneHotEncoder
from category_encoders.target_encoder import TargetEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, make_scorer
from scipy.stats import randint, uniform
from scipy.stats import ks_2samp

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [164]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [4]:
spark = (
    SparkSession.builder
        .appName('nathan_data_master_v1')
        .config("spark.driver.memory", "8g")  
        .config("spark.executor.memory", "20g")
        .config("spark.executor.cores", "4") 
        .config("spark.sql.shuffle.partitions", "400") 
        .config("spark.driver.maxResultSize", "2g")
        .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR") 

spark

In [78]:
def show(df, limit=20, truncate=True):
    from IPython.display import HTML
    data = df.limit(limit).collect()
    columns = df.columns

    html = '<div style="overflow-x: auto;"><table border="1"><thead><tr>'
    html += ''.join(f'<th>{col}</th>' for col in columns)
    html += '</tr></thead><tbody>'

    for row in data:
        html += '<tr>' + ''.join(f'<td>{str(cell)[:20] if truncate else cell}</td>' for cell in row) + '</tr>'

    html += '</tbody></table></div>'
    return HTML(html)

# SAMPLE

In [11]:
members = spark.read.parquet('members.parquet')
user_logs = spark.read.parquet('user_logs.parquet')
transactions = spark.read.parquet('transactions.parquet')

In [12]:
janela = Window.partitionBy("msno").orderBy("safra")

df = (members
    .join(user_logs, ['msno','safra'])
    .join(transactions, ['msno','safra'])
    .withColumn("safra_date", F.to_date(F.concat(F.col("safra").substr(1, 4), F.lit("-"), F.col("safra").substr(5, 2), F.lit("-01")), "yyyy-MM-dd"))
    .withColumn("safra_m3_date", F.add_months(F.col("safra_date"), 3))
    .withColumn("safra_m3", F.date_format(F.col("safra_m3_date"), "yyyyMM")) 
    
    .withColumn("is_ativo_safra_m3", 
                  F.when(F.col("safra_m3") == F.lead("safra", 3).over(janela), F.lead("is_ativo", 3).over(janela))
                   .otherwise(F.lit(0)))       
    # Criação da target
    .withColumn('target', F.when(
          (F.col('is_ativo') == 1) &
          (F.col('is_ativo_safra_m3') == 0), 1
          ).otherwise(0)
                 )
    .filter(
        (F.col('safra')>='201601') &
        (F.col('safra')<='201609')
    )
    .withColumn(
    "safra_target",
    F.concat_ws("_", F.col("safra").cast("string"), F.col("target").cast("string"))
    )
           
           )

In [13]:
### Validação ###

In [100]:
# show(all_data.filter(F.col('msno') == '++OepqRK4wiYg4Chl+qqo7TrwM+i9KZc3Ez/Swbjjew=').orderBy('safra').select('msno','safra','safra_m3','is_ativo','is_ativo_safra_m3','target'))

msno,safra,safra_m3,is_ativo,is_ativo_safra_m3,target
++OepqRK4wiYg4Chl+qq,201601,201604,1,1,0
++OepqRK4wiYg4Chl+qq,201602,201605,1,0,1
++OepqRK4wiYg4Chl+qq,201603,201606,1,0,1
++OepqRK4wiYg4Chl+qq,201604,201607,1,0,1


In [110]:
# show(all_data.filter(F.col('msno') == '+gKcZkvuBqRfJJIOhG9BPrEjUdNoMEUX8Yoy+1XN2yw=').orderBy('safra').select('msno','safra','safra_m3','is_ativo','is_ativo_safra_m3','target'))

msno,safra,safra_m3,is_ativo,is_ativo_safra_m3,target
+gKcZkvuBqRfJJIOhG9B,201601,201604,1,1,0
+gKcZkvuBqRfJJIOhG9B,201602,201605,1,1,0
+gKcZkvuBqRfJJIOhG9B,201603,201606,1,1,0
+gKcZkvuBqRfJJIOhG9B,201604,201607,1,1,0
+gKcZkvuBqRfJJIOhG9B,201605,201608,1,1,0
+gKcZkvuBqRfJJIOhG9B,201606,201609,1,1,0
+gKcZkvuBqRfJJIOhG9B,201607,201610,1,1,0
+gKcZkvuBqRfJJIOhG9B,201608,201611,1,1,0
+gKcZkvuBqRfJJIOhG9B,201609,201612,1,1,0


In [ ]:
### Validação ###

In [17]:
total = df.count()

In [181]:
total

5890982

In [192]:
# Supondo que você já tenha o total de linhas
total = df.count()

df.groupBy("safra").agg(
    F.count("*").alias("qtd"),
    F.round((F.count("*") / total) * 100, 1).alias("porcentagem_qtd"),
    F.round((F.sum("target") / F.count("*")) * 100, 1).alias("pct_target")
).orderBy("safra").show()

+------+------+---------------+----------+
| safra|   qtd|porcentagem_qtd|pct_target|
+------+------+---------------+----------+
|201601|647110|           11.0|      35.0|
|201602|598819|           10.2|      26.6|
|201603|593901|           10.1|      23.3|
|201604|595461|           10.1|      22.2|
|201605|605933|           10.3|      21.6|
|201606|620331|           10.5|      22.1|
|201607|721355|           12.2|      21.2|
|201608|748647|           12.7|      21.5|
|201609|759425|           12.9|      22.7|
+------+------+---------------+----------+



In [18]:
combinacoes = [r["safra_target"] for r in df.select("safra_target").distinct().collect()]

In [19]:
frac = 0.1  # 20% de cada grupo
fractions = {key: frac for key in combinacoes}

In [20]:
df_amostra = df.sampleBy("safra_target", fractions, seed=42).drop("safra_target")

In [204]:
# Supondo que você já tenha o total de linhas
total_amostra = df_amostra.count()

df_amostra.groupBy("safra").agg(
    F.count("*").alias("qtd"),
    F.round((F.count("*") / total_amostra) * 100, 1).alias("porcentagem_qtd"),
    F.round((F.sum("target") / F.count("*")) * 100, 1).alias("pct_target")
).orderBy("safra").show()

+------+-----+---------------+----------+
| safra|  qtd|porcentagem_qtd|pct_target|
+------+-----+---------------+----------+
|201601|65228|           11.1|      35.1|
|201602|59637|           10.1|      26.6|
|201603|59510|           10.1|      23.5|
|201604|59472|           10.1|      22.0|
|201605|60629|           10.3|      21.7|
|201606|61943|           10.5|      22.1|
|201607|72079|           12.2|      21.4|
|201608|74390|           12.6|      21.4|
|201609|76122|           12.9|      22.7|
+------+-----+---------------+----------+



In [21]:
features = ['registration_init_time',
 'city',
 'bd',
 'gender',
 'registered_via',
 'num_25',
 'num_50',
 'num_75',
 'num_985',
 'num_100',
 'num_unq',
 'total_secs',
 'payment_method_id',
 'payment_plan_days',
 'plan_list_price',
 'actual_amount_paid',
 'is_auto_renew',
 'transaction_date',
 'membership_expire_date',         
           ]

In [22]:
target = 'target'

In [23]:
train = df_amostra.filter(F.col('safra').between('201601','201606')).toPandas()
test = df_amostra.filter(F.col('safra').between('201607','201608')).toPandas()
oot =  df_amostra.filter(F.col('safra') == '201609').toPandas()

In [90]:
train['msno'].count()

365734

In [102]:
tx_test = ( train['msno'].count() / (train['msno'].count()+test['msno'].count()) )*100

In [120]:
print(f'Taxa Teste {tx_test:.0f}%')

Taxa Teste 71%


In [24]:
# Treino
X_train = train[features]
y_train = train[target]

# Teste
X_test = test[features]
y_test = test[target]

# Out-of-time
X_oot = oot[features]
y_oot = oot[target]

In [124]:
df_amostra.select(
    (F.mean('target'))
).show()

+-------------------+
|        avg(target)|
+-------------------+
|0.23988557070338365|
+-------------------+



In [26]:
print("Taxa variável resposta Treino:", y_train.mean())
print("Taxa variável resposta Test:", y_test.mean())
print("Taxa variável resposta OOT:", y_oot.mean())

Taxa variável resposta Treino: 0.25265083366599767
Taxa variável resposta Test: 0.21483048262116375
Taxa variável resposta OOT: 0.22883304213029257


In [ ]:
All_Data_Master/Tentativa_2/Data_master_tentativa2.ipynb

In [32]:
X_train.to_csv(f"C:/Users/natha/X_train_tent2.csv", index=False)
y_train.to_csv(f"C:/Users/natha/y_train_tent2.csv", index=False)
print(f'parte master salva')

parte master salva


# EXPLORE

In [82]:
X_train_sp = spark.read.csv("C:/Users/natha/X_train_tent2.csv", header=True, inferSchema=True)
y_train_sp = spark.read.csv("C:/Users/natha/y_train_tent2.csv", header=True, inferSchema=True)

In [369]:
X_train = pd.read_csv("C:/Users/natha/X_train_tent2.csv")
y_train = pd.read_csv("C:/Users/natha/y_train_tent2.csv")

## Nullos

In [72]:
X_train.isna().sum().sort_values(ascending=False)

gender                    182891
registration_init_time         0
num_unq                        0
transaction_date               0
is_auto_renew                  0
actual_amount_paid             0
plan_list_price                0
payment_plan_days              0
payment_method_id              0
total_secs                     0
num_100                        0
city                           0
num_985                        0
num_75                         0
num_50                         0
num_25                         0
registered_via                 0
bd                             0
membership_expire_date         0
dtype: int64

In [ ]:
# missing genero

## sumario

In [371]:
dummies = pd.get_dummies(X_train['gender'], prefix='gender', dummy_na=True).astype(int)
X_train = pd.concat([X_train.drop(columns='gender'), dummies], axis=1)

In [373]:
df_analise = X_train.copy()
df_analise[target] = y_train

In [323]:
# dummies = pd.get_dummies(df_analise['gender'], prefix='gender', dummy_na=True).astype(int)
# df_analise = pd.concat([df_analise, dummies], axis=1).drop(columns='gender')

In [403]:
feat=['city', 'bd', 'registered_via', 'num_25',
       'num_50', 'num_75', 'num_985', 'num_100', 'num_unq', 'total_secs',
       'payment_method_id', 'payment_plan_days', 'plan_list_price',
       'actual_amount_paid', 'is_auto_renew',
    'gender_female', 'gender_male',
       'gender_nan']

In [377]:
sumario = df_analise[feat+[target]].groupby(by=target).agg(['mean','median']).T

In [379]:
sumario['diff_abs'] = sumario[0] - sumario[1]
sumario['diff_rel'] = sumario[0] / sumario[1]

In [381]:
sumario.sort_values(by='diff_rel', ascending=False)

,target,0,1,diff_abs,diff_rel
gender_nan,median,1.00,0.00,1.00,inf
num_100,median,316.00,173.00,143.00,1.83
total_secs,median,86713.07,48953.60,37759.47,1.77
num_unq,median,355.00,204.00,151.00,1.74
is_auto_renew,mean,0.92,0.53,0.39,1.73
num_50,median,16.00,10.00,6.00,1.60
num_75,median,11.00,7.00,4.00,1.57
num_985,median,11.00,7.00,4.00,1.57
num_25,median,61.00,39.00,22.00,1.56
num_100,mean,541.01,395.04,145.97,1.37


## Arvore

In [407]:
from sklearn import tree
import matplotlib.pyplot as plt

In [439]:
arvore = tree.DecisionTreeClassifier(random_state=42, max_depth=4)
arvore_full = tree.DecisionTreeClassifier(random_state=42)

In [441]:
arvore.fit(X_train[feat], y_train)
arvore_full.fit(X_train[feat], y_train)

DecisionTreeClassifier(random_state=42)

In [442]:
plt.figure(figsize=(30, 20), dpi=500)

tree.plot_tree(
    arvore,
    feature_names=X_train.columns,
    class_names=list(map(str, arvore.classes_)),
    filled=True,        
    rounded=True,        
    fontsize=12,        
    precision=2        
)
plt.show()

In [443]:
feature_importance = (
    pd.Series(arvore_full.feature_importances_, 
                               index=X_train[feat].columns)
    .sort_values(ascending=False)
    .reset_index()
)

In [444]:
feature_importance['acumld'] = feature_importance[0].cumsum()

In [445]:
feature_importance

,index,0,acumld
0,is_auto_renew,0.19,0.19
1,total_secs,0.19,0.38
2,num_unq,0.10,0.48
3,payment_method_id,0.10,0.58
4,num_25,0.08,0.66
5,num_100,0.07,0.73
6,num_50,0.06,0.78
7,num_985,0.05,0.83
8,num_75,0.05,0.88
9,bd,0.03,0.91
